In [13]:
!pip install langchain[all] --upgrade
# Version: 0.0.164

!pip install pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.3.25 requires typing-extensions>=4.5.0, but you have typing-extensions 4.4.0 which is incompatible.



     -------------------------------------- 387.9/387.9 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 10.8/10.8 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'don

In [1]:
import sys

In [2]:
import langchain

In [3]:
from langchain.llms import OpenAI

In [5]:
from langchain.document_loaders import DirectoryLoader

In [6]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

### Load your data

In [7]:
loader = PyPDFLoader("../data/field-guide-to-data-science.pdf")

## Other options for loaders 
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [8]:
data = loader.load()

In [31]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 126 document(s) in your data
There are 2812 characters in your document


### Chunk your data up into smaller documents

In [10]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [11]:
print (f'Now you have {len(texts)} documents')

Now you have 162 documents


### Create embeddings of your documents to get ready for semantic search

In [14]:
!pip install pinecone-client

  Using cached pinecone_client-2.2.1-py3-none-any.whl (177 kB)
  Using cached loguru-0.7.0-py3-none-any.whl (59 kB)
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)


In [24]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\billb\AppData\Local\Programs\Python\Python39\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [16]:
import openai

In [18]:
%env OPENAI_API_KEY = 4fddda4393524f779892c9dfbbdbe00b 

env: OPENAI_API_KEY=4fddda4393524f779892c9dfbbdbe00b


In [19]:
%env AZURE_OPENAI_ENDPOINT = https://unisystemsopenai.openai.azure.com/ 

env: AZURE_OPENAI_ENDPOINT=https://unisystemsopenai.openai.azure.com/


In [20]:
import os
openai.api_type = "azure"
openai.api_base = "https://unisystemsopenai.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
# Check to see if there is an environment variable with you API keys, if not, use what you put below
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', '4fddda4393524f779892c9dfbbdbe00b ')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'bfbdb19b-5f3e-4899-91bc-5aa6b42e8f89')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-west4-gcp-free') # You may need to switch with your env

### Issue with Azure OpenAI

https://github.com/hwchase17/langchain/issues/1560

In [36]:
embeddings = OpenAIEmbeddings(chunk_size=1,openai_api_key=OPENAI_API_KEY)
#embeddings = OpenAIEmbeddings(chunk_size=1,openai_api_key=OPENAI_API_KEY)

In [37]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchaintest" # put in the name of your pinecone index here

In [34]:
!pip install tiktoken

                                              0.0/635.6 kB ? eta -:--:--
     ---                                     61.4/635.6 kB 1.1 MB/s eta 0:00:01
     -------                                122.9/635.6 kB 1.4 MB/s eta 0:00:01
     ------------                           204.8/635.6 kB 1.4 MB/s eta 0:00:01
     -----------------                      286.7/635.6 kB 1.4 MB/s eta 0:00:01
     ------------------------               409.6/635.6 kB 1.7 MB/s eta 0:00:01
     ---------------------------            460.8/635.6 kB 1.5 MB/s eta 0:00:01
     ----------------------------------     573.4/635.6 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 635.6/635.6 kB 1.7 MB/s eta 0:00:00
  Using cached regex-2023.5.5-cp39-cp39-win_amd64.whl (267 kB)


### Issue with Azure OpenAI


https://github.com/hwchase17/langchain/issues/1560

In [38]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [39]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query)

In [40]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

Intelligence and cloud infrastructure development  
work. We saw the need for a  
new approach to distill value 
from our clients’ data. We 
approached the problem 
with a multidisciplinary 
team of computer scientists, 
mathematicians and domain 
experts. They immediately 
produced new insights and 
analysis paths, solidifying the 
validity of the approach. Since 
that time, our Data Science  
team has grown to 250 staff 
supporting dozens of cl


### Query those docs to get your answer back

In [41]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [54]:
llm = OpenAI(temperature=0, engine="text-davinci-003",openai_api_key=OPENAI_API_KEY)
# llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [55]:
query = "What is the collect stage of data maturity?"
docs = docsearch.similarity_search(query)

In [56]:
chain.run(input_documents=docs, question=query)

' The collect stage of data maturity focuses on collecting internal or external datasets. Gathering sales records and corresponding weather data is an example of the collect stage.'